In [1]:
%matplotlib widget
%matplotlib inline
import sys
from sys import platform
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.ticker import ScalarFormatter
import numpy as np
import seaborn as sns
import scipy
from IPython.display import display, Math
from tqdm import tqdm

from astropy.io import fits, ascii
from astropy.table import Table
from astropy.modeling import functional_models, fitting
from astropy.timeseries import LombScargle

import stingray.events as ev
import stingray.lightcurve as lc
from stingray import io
import stingray.powerspectrum as powspec 
import stingray.crossspectrum as crossspec
import stingray.gti as sting_gti
import stingray.pulse.pulsar as plsr
import stingray.deadtime.fad as fad
from stingray import stats


sns.set_context('talk')
# sns.set_style("whitegrid")
sns.set_palette("colorblind")

sys.path.insert(1, '/Users/sean/scripts/helpers')

from stingray_plus import *



/Users/sean/opt/anaconda3/lib/python3.8/site-packages/stingray/utils.py:25: UserWarning: pyfftw not installed. Using standard scipy fft
  warnings.warn("pyfftw not installed. Using standard scipy fft")


# NuSTAR 90801312001

## Setup

In [2]:
if platform=='linux' or platform=='linux2':
    print('Working on SRL server')
    root_dir = '/disk/lif2/spike/1A_1744m361/'
elif platform=='darwin':
    print('Working on Macbook')
    root_dir = '/Volumes/Samsung_1TB/AstroData/1A_1744m361/'
    
OBSID='90801312001'

products_dir = root_dir + OBSID + '_pipe_out/'
plot_dir = root_dir + 'figures/'


Working on Macbook


## PDS and LS analysis

In [3]:
keV_min = 3
keV_max = 20
PI_min = int(eV_to_PI(keV_min * 1000))
PI_max = int(eV_to_PI(keV_max * 1000))


### Create PDS

events = extract_events(products_dir + 'nu' + OBSID + 'A01_cl_bc_regfilt.evt', \
            products_dir + 'nu' + OBSID + 'B01_cl_bc_regfilt.evt')

# FAD analysis

In [6]:
# plt.ion()
# bin_time = 1/2048
# split_time = 2048
# f_res = 0.1

# fad_results = fad.FAD(events[0], events[1], segment_size=split_time, norm='leahy', dt = bin_time, return_objects=True)
# fad_pds = fad_results['ptot'].rebin_log(f=f_res)

# fad_pds_log_err = fad_pds.df*np.power(1.+f_res, range(len(fad_pds.freq)))/2.



# plt.figure(figsize=(9,6))
# plt.errorbar(fad_results['freq'], fad_results['ptot'], xerr=fad_results.meta['df']/2, \
#              yerr=fad_results['ptot']/np.sqrt(fad_results.meta['M']), fmt='none', color='black', lw=0.5, rasterized=True)
# plt.step(np.concatenate([fad_results['freq']-fad_results.meta['df']/2, [fad_results['freq'][-1]+fad_results.meta['df']/2]]), \
#          np.concatenate([fad_results['ptot'], [fad_results['ptot'][-1]]]), where='post', color='black', lw=0.5, rasterized=True)
# plt.xscale('log')
# # plt.ylim(bottom=1e-3)
# # plt.yscale('log')
# plt.xlabel('Frequency (Hz)')
# # plt.ylabel(r'$\mathrm{Power\ (rms/mean)^{2}/Hz}$')
# plt.ylabel('Leahy Power')
# det_level_1sigma = stats.pds_detection_level(epsilon=1-0.6827, ntrial=len(fad_results['freq']), n_summed_spectra=fad_results.meta['M'])
# det_level_2sigma = stats.pds_detection_level(epsilon=1-0.9545, ntrial=len(fad_results['freq']), n_summed_spectra=fad_results.meta['M'])
# det_level_3sigma = stats.pds_detection_level(epsilon=1-0.9973, ntrial=len(fad_results['freq']), n_summed_spectra=fad_results.meta['M'])
# plt.axhline(det_level_1sigma, color='red', ls='dashed')
# plt.axhline(det_level_2sigma, color='purple', ls='dashed')
# plt.axhline(det_level_3sigma, color='blue', ls='dashed')
# plt.tight_layout()
# # plt.show()
# plt.savefig(plot_dir + 'FAD_PDS_Pnu_' + str(int(split_time)) + 's_' + str(keV_min)+'_'+str(keV_max) + 'keV_leahy.pdf')
# plt.close()


plt.figure(figsize=(9,6))
plt.errorbar(fad_pds.freq, fad_pds.power, yerr=fad_pds.power/np.sqrt(fad_pds.m), \
             fmt='none', color='black', lw=0.5)
plt.step(fad_pds.freq-fad_pds_log_err, fad_pds.power, where='post', color='black', lw=0.5)
plt.xscale('log')
# plt.ylim(bottom=1e-3)
# plt.yscale('log')
plt.xlabel('Frequency (Hz)')
# plt.ylabel(r'$\mathrm{Power\ (rms/mean)^{2}/Hz}$')
plt.ylabel('Leahy Power')
det_level_1sigma = stats.pds_detection_level(epsilon=1-0.6827, ntrial=len(fad_results['ptot'].freq), n_rebin=fad_pds.m)
det_level_2sigma = stats.pds_detection_level(epsilon=1-0.9545, ntrial=len(fad_results['ptot'].freq), n_rebin=fad_pds.m)
det_level_3sigma = stats.pds_detection_level(epsilon=1-0.9973, ntrial=len(fad_results['ptot'].freq), n_rebin=fad_pds.m)
# plt.axhline(det_level_1sigma, color='red', ls='dashed')
# plt.axhline(det_level_2sigma, color='purple', ls='dashed')
# plt.axhline(det_level_3sigma, color='blue', ls='dashed')
# plt.plot(fad_pds.freq-fad_pds.df/2, det_level_1sigma, color='red', ls='dashed')
# plt.plot(fad_pds.freq-fad_pds.df/2, det_level_2sigma, color='purple', ls='dashed')
plt.step(np.concatenate([[fad_pds.freq[0]-fad_pds_log_err[0]], fad_pds.freq]), np.concatenate([[det_level_3sigma[0]], det_level_3sigma]), where='mid',color='blue', ls='dashed', lw=0.6)
plt.tight_layout()
plt.loglog()
# plt.show()
plt.savefig(plot_dir + 'FAD_PDS_Pnu_' + str(int(split_time)) + 's_' + str(keV_min)+'_'+str(keV_max) + 'keV_leahy.pdf')
plt.close()



In [5]:
print(fad_results.meta['M'])
print(fad_pds)

13
       ptot       
------------------
 6.876617131992895
 6.581071758041617
 2.616297483164932
5.2458685043362925
3.2311987676860454
3.9164411601502325
3.3142591226670217
               ...
 2.553222565667059
 2.245285981419334
  1.43943403386814
 2.677986171185498
 2.497938746224432
2.2759929847204905
1.3203311222430894
Length = 524287 rows


In [7]:
f_arr, z_prob, z_stat = efold_search_AandB(events[0], events[1], f_min=520, f_max=540, f_steps=100000, nbin = 32, pi_min = PI_min, pi_max = PI_max)

# popt, pcov = scipy.optimize.curve_fit(gauss, 1/f_arr, z_stat, p0 = [100,322.7,0.5])
# print(popt)
# print(np.sqrt(np.diag(pcov)))

plt.figure(figsize= (9,6))
plt.plot(f_arr, z_stat)
# plt.plot(1/f_arr, gauss(1/f_arr, *popt))
print(np.max(z_prob))
print(f_arr[np.argmax(z_prob)])

sorted_prob = np.argsort((1-z_prob))
summed = np.cumsum((1-z_prob)[sorted_prob])
print(f_arr[sorted_prob[np.argmin(summed-0.32)]])
print(np.min([np.min(events[0].time), np.min(events[1].time)]))



TypeError: 'NoneType' object is not subscriptable